In [1]:
import pysam
import collections
from math import floor, ceil
from pprint import pprint
import sys
import functions 
import pandas as pd

In [2]:
#lectura del archivo bam 
bam_file = pysam.AlignmentFile('SRR769610_trim.bam','rb')
#header - contiene informacion sobre el alineamiento y los cromosomas
header = bam_file.header.to_dict()
total_reads = bam_file.mapped

#obtencion de reads e informacion por cromosoma
chromosomes = {}
genome_size = 0
bin_pos = {}

for i in header['SQ']:
    chromosome_name = i['SN']
    chromosome_lenght = i['LN']
    genome_size += chromosome_lenght


    #chromosomes[i['SN']] =  #para sacar los reads de un cromosoma se usa fecth
    #i['LN'] es el largo
    reads = bam_file.fetch(i['SN'])
    
     #lista de posiciones de los reads
    pos = []
    for read in reads:
        p = (read.to_dict())['ref_pos']
        pos.append(p)

    #obtencion de bins 
    bins = [functions.pos_to_bin(p) for p in pos]
    o_bins = [functions.pos_to_bin(p, overlap=True) for p in pos]
    c = dict(collections.Counter(bins + o_bins))

    bin_pos[chromosome_name] = [pos]
    
    chromosomes[chromosome_name] = dict(sorted(c.items()))


bam_file.close()


In [3]:
for k,v in chromosomes.items():

    dict_aux = {}
    bins = list(v.keys())
    counts = list(v.values())
    
    for i in range(len(v)):
        dict_aux[bins[i]] = functions.bin_to_RPM(counts[i], total_reads) 
        #print(bins[i], counts[i])
    
    chromosomes[k] = dict_aux

In [4]:
chromosomes_bin = {}
for chr_name, values in chromosomes.items():
  #print(chr_name, values)
  bins = functions.bin_group(values, treshold= 2)
  if len(bins) != 0 :
    chromosomes_bin[chr_name] = functions.clustering_bins(bins, treshold = 2)
 #   test_clust = functions.clustering_bins(test, treshold = 2)

In [5]:
#number of clusters
cluster_density = {}
num_clusters = {}

total_clusters = 0
for k,v in chromosomes_bin.items():

    num_cluster = len(v)
    total_clusters += num_cluster    #obtencion de bins 
    bins = [functions.pos_to_bin(p) for p in pos]
    o_bins = [functions.pos_to_bin(p, overlap=True) for p in pos]
    c = dict(collections.Counter(bins + o_bins))

    bin_pos[chromosome_name] = [pos]
    
    chromosomes[chromosome_name] = dict(sorted(c.items()))
    cluster_density_chr = num_cluster/genome_size

    num_clusters[k] = num_cluster
    cluster_density[k] = cluster_density_chr
    
num_clusters['total_clusters'] = total_clusters
cluster_density['cluster density global'] = total_clusters/genome_size
print('cluster density global = {}'.format(total_clusters/genome_size))

cluster density global = 0.00029674682082871215


In [6]:
df_cluster_density = pd.DataFrame.from_dict(cluster_density, orient='index')
df_cluster_density.to_csv("cluster_density.tsv", sep='\t')

df_cluster_num = pd.DataFrame.from_dict(num_clusters, orient='index')
df_cluster_num.to_csv("cluster_num.tsv", sep='\t')


In [24]:
#cluster sizes
cluster_size = {}

for k,v in chromosomes_bin.items():
    clusters = []
    for n in range(len(v)):
        l = (v[n][-1] - v[n][0])
        
        try:

            if ((l+1)%2) == 0:
                start = int((v[n][0]/2)*50)
                stop = int(((v[n][-1]/2)*50)+50)
                inter_space = int((v[n+1][0]/2)*50) - stop
                print("start:{} {} pb, stop: {} {} pb, size: {}, lenght: {}, interspace: {}".format(v[n][0], start, v[n][-1], stop, (l+1), (l+2)*25, inter_space ))
                
                cluster_data = {'cluster':[v[n][0], v[n][-1]], 'start': start, 'stop': stop, 'lenght': (l+2)*25, 'size' : (l+1), 'distance_next_cluster': inter_space}

                clusters.append(cluster_data)        

            else:
                
                start = int((v[n][0]/2)*50)
                stop = int(((v[n][-1]/2)*50)+50)
                inter_space = int((v[n+1][0]/2)*50) - stop

                print("start:{} {} pb, stop: {} {} pb, size: {}, lenght: {}, interspace: {}".format(v[n][0], start, v[n][-1], stop, (l+1), (ceil((l+1)/2))*50, inter_space))
                
                cluster_data = {'cluster':[v[n][0], v[n][-1]], 'start': start, 'stop': stop, 'lenght': (ceil((l+1)/2))*50, 'size' : (l+1), 'distance_next_cluster': inter_space}

                clusters.append(cluster_data)

        except IndexError:
            
            if ((l+1)%2) == 0:
                start = int((v[n][0]/2)*50)
                stop = int(((v[n][-1]/2)*50)+50)
                inter_space = 0
                print("start:{} {} pb, stop: {} {} pb, size: {}, lenght: {}, interspace: {}".format(v[n][0], start, v[n][-1], stop, (l+1), (l+2)*25, inter_space ))
                
                cluster_data = {'cluster':[v[n][0], v[n][-1]], 'start': start, 'stop': stop, 'lenght': (l+2)*25, 'size' : (l+1), 'distance_next_cluster': inter_space}

                clusters.append(cluster_data)        

            else:
                
                start = int((v[n][0]/2)*50)
                stop = int(((v[n][-1]/2)*50)+50)
                inter_space = 0

                print("start:{} {} pb, stop: {} {} pb, size: {}, lenght: {}, interspace: {}".format(v[n][0], start, v[n][-1], stop, (l+1), (ceil((l+1)/2))*50, inter_space))
                
                cluster_data = {'cluster':[v[n][0], v[n][-1]], 'start': start, 'stop': stop, 'lenght': (ceil((l+1)/2))*50, 'size' : (l+1), 'distance_next_cluster': inter_space}

                clusters.append(cluster_data)

    cluster_size[k] = clusters

start:8 200 pb, stop: 8 250 pb, size: 1, lenght: 50, interspace: 750
start:40 1000 pb, stop: 41 1075 pb, size: 2, lenght: 75, interspace: 150
start:49 1225 pb, stop: 59 1525 pb, size: 11, lenght: 300, interspace: 50
start:63 1575 pb, stop: 74 1900 pb, size: 12, lenght: 325, interspace: 175
start:83 2075 pb, stop: 100 2550 pb, size: 18, lenght: 475, interspace: 25
start:103 2575 pb, stop: 114 2900 pb, size: 12, lenght: 325, interspace: 25
start:117 2925 pb, stop: 119 3025 pb, size: 3, lenght: 100, interspace: 75
start:124 3100 pb, stop: 125 3175 pb, size: 2, lenght: 75, interspace: 25
start:128 3200 pb, stop: 129 3275 pb, size: 2, lenght: 75, interspace: 75
start:134 3350 pb, stop: 136 3450 pb, size: 3, lenght: 100, interspace: 50
start:140 3500 pb, stop: 141 3575 pb, size: 2, lenght: 75, interspace: 50
start:145 3625 pb, stop: 156 3950 pb, size: 12, lenght: 325, interspace: 50
start:160 4000 pb, stop: 161 4075 pb, size: 2, lenght: 75, interspace: 50
start:165 4125 pb, stop: 166 4200 pb

In [29]:
pd.DataFrame.from_dict({(i,j): cluster_size[i][j] 
                           for i in cluster_size.keys() 
                           for j in cluster_size[i].keys()},
                       orient='index')

AttributeError: 'list' object has no attribute 'keys'

In [27]:
df_cluster_size = pd.DataFrame.from_dict(cluster_size, orient='index')
#df_cluster_num.to_csv("cluster_num.tsv", sep='\t')

In [28]:
df_cluster_size

,0,1,2,3,4,5,6,7,8,9,...,2616,2617,2618,2619,2620,2621,2622,2623,2624,2625
I,"{'cluster': [8, 8], 'start': 200, 'stop': 250,...","{'cluster': [40, 41], 'start': 1000, 'stop': 1...","{'cluster': [49, 59], 'start': 1225, 'stop': 1...","{'cluster': [63, 74], 'start': 1575, 'stop': 1...","{'cluster': [83, 100], 'start': 2075, 'stop': ...","{'cluster': [103, 114], 'start': 2575, 'stop':...","{'cluster': [117, 119], 'start': 2925, 'stop':...","{'cluster': [124, 125], 'start': 3100, 'stop':...","{'cluster': [128, 129], 'start': 3200, 'stop':...","{'cluster': [134, 136], 'start': 3350, 'stop':...",...,"{'cluster': [391811, 391812], 'start': 9795275...","{'cluster': [391827, 391828], 'start': 9795675...","{'cluster': [391833, 391838], 'start': 9795825...","{'cluster': [391848, 391851], 'start': 9796200...","{'cluster': [391858, 391872], 'start': 9796450...","{'cluster': [391878, 391886], 'start': 9796950...","{'cluster': [391890, 391897], 'start': 9797250...","{'cluster': [391901, 391902], 'start': 9797525...","{'cluster': [391906, 391911], 'start': 9797650...","{'cluster': [391916, 391930], 'start': 9797900..."
Mt,"{'cluster': [-1, 12], 'start': -25, 'stop': 35...","{'cluster': [15, 16], 'start': 375, 'stop': 45...","{'cluster': [20, 23], 'start': 500, 'stop': 62...","{'cluster': [26, 33], 'start': 650, 'stop': 87...","{'cluster': [40, 40], 'start': 1000, 'stop': 1...","{'cluster': [47, 70], 'start': 1175, 'stop': 1...","{'cluster': [75, 75], 'start': 1875, 'stop': 1...","{'cluster': [78, 90], 'start': 1950, 'stop': 2...","{'cluster': [93, 108], 'start': 2325, 'stop': ...","{'cluster': [115, 118], 'start': 2875, 'stop':...",...,None,None,None,None,None,None,None,None,None,None
II,"{'cluster': [18, 20], 'start': 450, 'stop': 55...","{'cluster': [28, 31], 'start': 700, 'stop': 82...","{'cluster': [37, 38], 'start': 925, 'stop': 10...","{'cluster': [43, 44], 'start': 1075, 'stop': 1...","{'cluster': [48, 58], 'start': 1200, 'stop': 1...","{'cluster': [61, 62], 'start': 1525, 'stop': 1...","{'cluster': [70, 76], 'start': 1750, 'stop': 1...","{'cluster': [79, 85], 'start': 1975, 'stop': 2...","{'cluster': [89, 90], 'start': 2225, 'stop': 2...","{'cluster': [95, 104], 'start': 2375, 'stop': ...",...,None,None,None,None,None,None,None,None,None,None
III,"{'cluster': [7, 9], 'start': 175, 'stop': 275,...","{'cluster': [12, 14], 'start': 300, 'stop': 40...","{'cluster': [27, 30], 'start': 675, 'stop': 80...","{'cluster': [37, 37], 'start': 925, 'stop': 97...","{'cluster': [44, 45], 'start': 1100, 'stop': 1...","{'cluster': [54, 54], 'start': 1350, 'stop': 1...","{'cluster': [58, 62], 'start': 1450, 'stop': 1...","{'cluster': [69, 70], 'start': 1725, 'stop': 1...","{'cluster': [78, 79], 'start': 1950, 'stop': 2...","{'cluster': [84, 85], 'start': 2100, 'stop': 2...",...,None,None,None,None,None,None,None,None,None,None
IV,"{'cluster': [21, 23], 'start': 525, 'stop': 62...","{'cluster': [26, 27], 'start': 650, 'stop': 72...","{'cluster': [31, 49], 'start': 775, 'stop': 12...","{'cluster': [52, 86], 'start': 1300, 'stop': 2...","{'cluster': [91, 93], 'start': 2275, 'stop': 2...","{'cluster': [97, 101], 'start': 2425, 'stop': ...","{'cluster': [104, 105], 'start': 2600, 'stop':...","{'cluster': [110, 115], 'start': 2750, 'stop':...","{'cluster': [119, 121], 'start': 2975, 'stop':...","{'cluster': [124, 128], 'start': 3100, 'stop':...",...,None,None,None,None,None,None,None,None,None,None
V,"{'cluster': [22, 23], 'start': 550, 'stop': 62...","{'cluster': [207, 208], 'start': 5175, 'stop':...","{'cluster': [247, 253], 'start': 6175, 'stop':...","{'cluster': [269, 289], 'start': 6725, 'stop':...","{'cluster': [1320, 1321], 'start': 33000, 'sto...","{'cluster': [1330, 1331], 'start': 33250, 'sto...","{'cluster': [2369, 2370], 'start': 59225, 'sto...","{'cluster': [2391, 2392], 'start': 59775, 'sto...","{'cluster': [2398, 2398], 'start': 59950, 'sto...","{'cluster': [2401, 2402], 'start': 60025, 'sto...",...,None,None,None,None,None,None,None,None,None

In [24]:
#inter space lenght
for i in chromosomes_bin.values():
    for n in range(len(i)):
        try:
          l = (i[n+1][0] - i[n][-1])

          if ((l-1)%2) == 0:
            print("start:{}, stop: {}, size: {}, lenght: {}".format(i[n][-1], i[n+1][0], (l-1), ((l-2)*25)))

          else:
            print("start:{}, stop: {}, size: {}, lenght: {}".format(i[n][-1], i[n+1][0], (l-1), (floor((l-1)/2))*50))
        
        except:
          IndexError


start:8, stop: 40, size: 31, lenght: 750
start:41, stop: 49, size: 7, lenght: 150
start:59, stop: 63, size: 3, lenght: 50
start:74, stop: 83, size: 8, lenght: 175
start:100, stop: 103, size: 2, lenght: 25
start:114, stop: 117, size: 2, lenght: 25
start:119, stop: 124, size: 4, lenght: 75
start:125, stop: 128, size: 2, lenght: 25
start:129, stop: 134, size: 4, lenght: 75
start:136, stop: 140, size: 3, lenght: 50
start:141, stop: 145, size: 3, lenght: 50
start:156, stop: 160, size: 3, lenght: 50
start:161, stop: 165, size: 3, lenght: 50
start:166, stop: 169, size: 2, lenght: 25
start:176, stop: 179, size: 2, lenght: 25
start:180, stop: 187, size: 6, lenght: 125
start:187, stop: 190, size: 2, lenght: 25
start:199, stop: 202, size: 2, lenght: 25
start:204, stop: 207, size: 2, lenght: 25
start:209, stop: 217, size: 7, lenght: 150
start:218, stop: 221, size: 2, lenght: 25
start:222, stop: 230, size: 7, lenght: 150
start:241, stop: 246, size: 4, lenght: 75
start:260, stop: 267, size: 6, lengh

In [14]:
pd.DataFrame.from_dict(chromosomes_bin, orient='index')

,0,1,2,3,4,5,6,7,8,9,...,4977,4978,4979,4980,4981,4982,4983,4984,4985,4986
I,[8],[40],"[50, 52, 54, 56, 58]","[64, 66, 68, 70, 72, 74]","[84, 86, 88, 90, 92, 94, 96, 98, 100]","[104, 106, 108, 110, 112, 114]",[118],[124],[128],"[134, 136]",...,"[391849, 391851]",[391859],"[391863, 391865, 391867, 391869, 391871]","[391879, 391881, 391883, 391885]",[391891],"[391895, 391897]",[391901],"[391907, 391909, 391911]","[391917, 391919, 391921, 391923, 391925]",[391929]
Mt,"[0, 2, 4, 6, 8, 10, 12]",[16],"[20, 22]","[26, 28, 30, 32]",[40],"[48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70]","[78, 80, 82, 84, 86, 88, 90]","[94, 96, 98, 100, 102, 104, 106, 108]","[116, 118]","[122, 124]",...,None,None,None,None,None,None,None,None,None,None
II,"[18, 20]",[28],[38],[44],"[48, 50, 52, 54, 56, 58]",[62],[70],"[74, 76]","[80, 84]",[90],...,None,None,None,None,None,None,None,None,None,None
III,[8],"[12, 14]","[28, 30]",[44],[54],"[58, 60, 62]",[70],[78],[84],[88],...,None,None,None,None,None,None,None,None,None,None
IV,[22],[26],"[32, 34, 36, 38, 40, 42, 44]",[48],"[52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 7...",[86],[92],"[98, 100]",[104],"[110, 112, 114]",...,None,None,None,None,None,None,None,None,None,None
V,[22],[208],"[248, 250, 252]","[270, 272, 274, 276, 278, 280, 282, 284, 286, ...",[1320],[1330],[2370],[2392],[2398],[2402],...,None,None,None,None,None,None,None,None,None,None
VI,[6],[32],"[44, 46]",[50],"[54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 7...","[90, 92, 94]","[98, 100, 102, 104, 106, 108, 110, 112]",[116],"[120, 122, 124, 126]","[130, 132, 134]",...,None,None,None,None,None,None,None,None,None,None
VII,[6],"[12, 14]","[22, 24]","[34, 36, 38, 40, 42, 44, 46, 48]",[56],"[60, 62]","[66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 8...","[96, 98]","[104, 106, 108, 110]",[118],...,None,None,None,None,None,None,None,None,None,None
supercont12.8,[2],[14],"[20, 22, 24]","[38, 40, 42]","[50, 52]",[68],"[88, 90]",[102],[106],"[110, 112]",...,None,None,None,None,None,None,None,None,None,None
supercont12.9,"[8, 10, 12, 14]","[18, 20]","[28, 30]",[36],[40],[46],"[52, 54, 56, 58, 60, 62, 64, 66]","[70, 72]","[76, 78]",[84],...,None,None,None,None,None,None,None,None,None,None


In [49]:
for i in list(chromosomes.keys()):
    if i in list(chromosomes_bin.keys()):
        
        print(i, len(chromosomes_bin[i]), len(bin_pos[i][0]))

I 2626 2283124
Mt 164 472245
II 1078 1504310
III 1544 2349973
IV 1898 3256254
V 1664 1875329
VI 1306 2057866
VII 1173 1591099
supercont12.8 315 9637209
supercont12.9 385 161643
supercont12.12 18 4151
supercont12.15 9 18061
supercont12.16 7 1769
supercont12.18 8 2631
supercont12.19 1 543
supercont12.20 1 378983


In [55]:
len(chromosomes_bin['I'])

2626

In [ ]:

print([functions.pos_to_bin(p) for p in bin_pos['I'][0]])


In [59]:
bin_pos['I'][0]

['195',
 '207',
 '207',
 '207',
 '207',
 '207',
 '207',
 '212',
 '216',
 '222',
 '222',
 '222',
 '223',
 '223',
 '224',
 '224',
 '225',
 '225',
 '225',
 '225',
 '226',
 '226',
 '226',
 '227',
 '227',
 '227',
 '227',
 '227',
 '227',
 '227',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '228',
 '229',
 '229',
 '231',
 '231',
 '232',
 '232',
 '233',
 '233',
 '234',
 '234',
 '234',
 '234',
 '234',
 '235',
 '235',
 '633',
 '651',
 '652',
 '957',
 '959',
 '959',
 '959',
 '960',
 '960',
 '960',
 '960',
 '962',
 '962',
 '1009',
 '1009',
 '1009',
 '1013',
 '1013',
 '1013',
 '1024',
 '1025',
 '1025',
 '1025',
 '1028',
 '1028',
 '1028',
 '1028',
 '1028',
 '1028',
 '1028',
 '1029',
 '1029',
 '1029',
 '1029',
 '1030',
 '1035',
 '1035',
 '1035',
 '1035',
 '1035',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '1036',
 '

In [ ]:
len(pos[10]['seq'])

24

In [19]:
pos[0]

{'name': 'SRR769610.25439498',
 'flag': '16',
 'ref_name': 'I',
 'ref_pos': '195',
 'map_quality': '255',
 'cigar': '19M',
 'next_ref_name': '*',
 'next_ref_pos': '0',
 'length': '0',
 'seq': 'TAACCCTAACCCTGCTCTA',
 'qual': '*',
 'tags': ['XA:i:0',
  'MD:Z:19',
  'NM:i:0',
  'XM:i:1',
  'RG:Z:SRR769610_trim',
  'XX:i:1',
  'XY:Z:U',
  'XZ:f:1']}